# Analysis of the error messages and creation of the word embeddings

In [1]:
import pandas as pd

In [2]:
import psutil

In [3]:
import numpy as np

In [4]:
import nltk
import pandas as pd
import numpy as np
import gensim
import itertools
from collections import Counter

In [5]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [6]:
from nltk.tokenize.treebank import TreebankWordTokenizer #word_tokenize
from nltk.tokenize import word_tokenize

In [4]:
import pickle
embedding_matrix = pickle.load( open( 'embedding_matrix.pkl', "rb" ) )

In [8]:
np.save('embedding_matrix.npy', embedding_matrix)

## 1. Read the data and clean up the text

In [5]:
#df_msg = pd.read_csv('/eos/user/l/llayer/AIErrorHandling/df_error_msg_exitcodes.csv')

In [7]:
test = pd.read_hdf('messages_filtered_20170101_20171009.h5')

In [3]:
psutil.cpu_percent(), psutil.virtual_memory()

(4.1,
 svmem(total=270201565184, available=222180610048, percent=17.8, used=42799063040, free=118444830720, active=104896872448, inactive=31373971456, buffers=5287936, cached=108952383488, shared=4316569600, slab=11605344256))

In [8]:
time_chunks =  [
                (20170101, 20171009),
                (20171011, 20180301),
                (20180301, 20180601),
                (20180601, 20181101),
                (20181101, 20190207),
                (20190207, 20190801),   
                ]

dfs = []
for time in time_chunks:
    t1, t2 = str(time[0]), str(time[1])
    frame = pd.read_hdf( 'messages_filtered_' + t1 + '_' + t2 + '.h5' )
    dfs.append( frame[['task_name', 'error',  'site', 'exit_codes', 'error_msg', 'error_type', 'steps_counter', 'names']])

In [9]:
dfs = pd.concat(dfs, ignore_index=True)

In [34]:
dfs = dfs.drop(columns=['readTotalSecs', 'NumberOfStreams', 'NumberOfThreads'])

KeyError: "['readTotalSecs' 'NumberOfStreams' 'NumberOfThreads'] not found in axis"

In [11]:
list(dfs)

['task_name',
 'error',
 'site',
 'exit_codes',
 'error_msg',
 'error_type',
 'steps_counter',
 'names']

In [10]:
dfs = dfs.drop_duplicates()

In [10]:
len(dfs)

610462

In [11]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
dfs.head(1000)

,task_name,error,site,exit_codes,error_msg,error_type,steps_counter,names
0,/fabozzi_Run2016F-v1-MuonEG-07Aug17_8029_17083...,85,T2_US_Purdue,99999,Could not find report file for step stageOut1!,ReportManipulatingError,0,stageOut1
1,/fabozzi_Run2016F-v1-MuonEG-07Aug17_8029_17083...,85,T2_US_Purdue,85,Adding last 25 lines of CMSSW stdout: 3 ...,CMSSWStepFailure,2,cmsRun1
2,/fabozzi_Run2016F-v1-MuonEG-07Aug17_8029_17083...,85,T2_US_Purdue,8021,An exception of category 'FileReadError' occur...,Fatal Exception,2,cmsRun1
3,/fabozzi_Run2016F-v1-MuonEG-07Aug17_8029_17083...,85,T2_US_Purdue,99999,Adding extra error in order to hold error repo...,ErrorLoggingAddition,2,cmsRun1
4,/fabozzi_Run2016G-v1-DoubleMuonLowMass-07Aug17...,85,T2_US_Florida,99999,Could not find report file for step stageOut1!,ReportManipulatingError,0,stageOut1
5,/fabozzi_Run2016G-v1-DoubleMuonLowMass-07Aug17...,85,T2_US_Florida,85,Adding last 25 lines of CMSSW stdout: 1476 ...,CMSSWStepFailure,2,cmsRun1
6,/fabozzi_Run2016G-v1-DoubleMuonLowMass-07Aug17...,85,T2_US_Florida,8021,An exception of category 'FileReadError' occur...,Fatal Exception,2,cmsRun1
7,/fabozzi_Run2016G-v1-DoubleMuonLowMass-07Aug17...,85,T2_US_Florida,8021,An exception of category 'FileReadError' occur...,Fatal Exception,2,cmsRun1
8,/fabozzi_Run2016G-v1-DoubleMuonLowMass-07Aug17...,85,T2_US_Florida,8021,An exception of category 'FileReadError' occur...,Fatal Exception,2,cmsRun1
9,/fabozzi_Run2016G-v1-DoubleMuonLowMass-07Aug17...,85,T2_US_Florida,99999,Adding extra error in order to hold error repo...,ErrorLoggingAddition,2,cmsRun1


In [10]:
dfs = dfs.drop_duplicates(subset=['task_name',
 'error',
 'site',
 'exit_codes',
 'error_type',
 'steps_counter',
 'names'])

In [14]:
len(dfs)

438866

In [42]:
test = dfs[dfs['error'] == dfs['exit_codes']]

In [45]:
test.head(100)

,task_name,error,site,exit_codes,error_msg,error_type,steps_counter,names,peakvaluerss,peakvaluevsize,writeTotalMB,readPercentageOps,readAveragekB,readTotalMB,readNumOps,readCachePercentageOps,readMBSec,writeTotalSecs,readMaxMSec,TotalJobCPU,TotalInitCPU,TotalEventCPU,AvgEventCPU,EventThroughput,TotalInitTime,AvgEventTime,MinEventCPU,MaxEventTime,TotalJobTime,TotalLoopCPU,MinEventTime,MaxEventCPU,_c0
1,/fabozzi_Run2016F-v1-MuonEG-07Aug17_8029_17083...,85,T2_US_Purdue,85,Adding last 25 lines of CMSSW stdout: 3 ...,CMSSWStepFailure,2,cmsRun1,0.00,0.000,4.365680,1.277778,60.032058,1.348376,18.0,0.0,0.422751,0.019534,767.755,115.963,-1.0,-1.0,-1.0,0.000000,-1.0,-1.000000,-1.0,0.000000,305.69700,105.51700,1000000.000000,-1.0,101202
5,/fabozzi_Run2016G-v1-DoubleMuonLowMass-07Aug17...,85,T2_US_Florida,85,Adding last 25 lines of CMSSW stdout: 1476 ...,CMSSWStepFailure,2,cmsRun1,3785.73,5511.360,564.784000,1.268293,4944.037285,1255.321967,205.0,0.0,15.438374,23.038100,3791.490,13486.700,-1.0,-1.0,-1.0,0.333684,-1.0,8.804770,-1.0,196.974000,5066.79000,13437.70000,2.288010,-1.0,336353
10,/fabozzi_Run2016G-v1-HTMHT-07Aug17_8029_170831...,50660,T0_CH_CERN,50660,Error in CMSSW step cmsRun1 Job has exceeded m...,PerformanceKill,0,PerformanceError,-1.00,-1.000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-1.000,-1.000,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.00000,-1.00000,-1.000000,-1.0,235600
11,/fabozzi_Run2016G-v1-HTMHT-07Aug17_8029_170831...,50660,T0_CH_CERN,50660,Error in CMSSW step cmsRun1 Job has exceeded m...,PerformanceKill,0,PerformanceError,-1.00,-1.000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-1.000,-1.000,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.00000,-1.00000,-1.000000,-1.0,235600
12,/fabozzi_Run2016G-v1-HTMHT-07Aug17_8029_170831...,50660,T0_CH_CERN,50660,Error in CMSSW step cmsRun1 Job has exceeded m...,PerformanceKill,0,PerformanceError,-1.00,-1.000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-1.000,-1.000,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.00000,-1.00000,-1.000000,-1.0,235600
13,/fabozzi_Run2016G-v1-HTMHT-07Aug17_8029_170831...,50660,T0_CH_CERN,50660,Error in CMSSW step cmsRun1 Job has exceeded m...,PerformanceKill,0,PerformanceError,-1.00,-1.000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-1.000,-1.000,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.00000,-1.00000,-1.000000,-1.0,235600
14,/fabozzi_Run2016G-v1-HTMHT-07Aug17_8029_170831...,50660,T0_CH_CERN,50660,Error in CMSSW step cmsRun1 Job has exceeded m...,PerformanceKill,0,PerformanceError,-1.00,-1.000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-1.000,-1.000,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.00000,-1.00000,-1.000000,-1.0,235600
17,/fabozzi_Run2016H-v1-DoubleEG-07Aug17_8029_170...,92,T2_UK_SGrid_RALPP,92,Adding last 25 lines of CMSSW stdout: 1536 ...,CMSSWStepFailure,2,cmsRun1,5348.79,7448.690,1668.670000,1.178503,3525.475798,2113.908340,521.0,0.0,61.155340,22.598000,3334.480,75534.600,-1.0,-1.0,-1.0,0.125563,-1.0,30.977600,-1.0,172.391000,22299.40000,75368.60000,4.050640,-1.0,101313
20,/fabozzi_Run2016H-v1-DoubleEG-07Aug17_8029_170...,50664,T2_IT_Rome,50664,Error in CMSSW step cmsRun1 Job has been runni...,PerformanceKill,0,PerformanceError,-1.00,-1.000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-1.000,-1.000,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.00000,-1.00000,-1.000000,-1.0,555
21,/fabozzi_Run2016H-v1-DoubleEG-07Aug17_8029_170...,50664,T2_IT_Rome,50664,Error in CMSSW step cmsRun1 Job has been runni...,PerformanceKill,0,PerformanceError,-1.00,-1.000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-1.000,-1.000,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.00000,-1.00000,-1.000000,-1.0,555


In [48]:
test.error_type.value_counts()

PerformanceKill                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [14]:
len(dfs.groupby(['task_name', 'error', 'site']))

117672

In [36]:
for col in list(dfs):
    if len(dfs[col].unique()) < 15:
        print col
        print dfs[col].unique()

steps_counter
[0 2 1 3 4 5 6 7]
names
[u'stageOut1' u'cmsRun1' u'PerformanceError' u'logArch1' u'cmsRun2'
 u'upload1' u'JobKilled' u'cmsRun3' u'RemovedByGLIDEIN' u'NoJobReport'
 u'CRITICAL' u'cmsRun4' u'cmsRun5']


In [26]:
print list(dfs)

['task_name', 'error', 'site', 'exit_codes', 'error_msg', 'error_type', 'steps_counter', 'names', 'peakvaluerss', 'peakvaluevsize', 'writeTotalMB', 'readPercentageOps', 'readAveragekB', 'readTotalMB', 'readNumOps', 'readCachePercentageOps', 'readMBSec', 'writeTotalSecs', 'readTotalSecs', 'readMaxMSec', 'TotalJobCPU', 'NumberOfStreams', 'TotalInitCPU', 'TotalEventCPU', 'AvgEventCPU', 'EventThroughput', 'TotalInitTime', 'AvgEventTime', 'NumberOfThreads', 'MinEventCPU', 'MaxEventTime', 'TotalJobTime', 'TotalLoopCPU', 'MinEventTime', 'MaxEventCPU', '_c0']


In [12]:
df_msg = dfs

In [11]:
# Tokenize the text
t = TreebankWordTokenizer()
dfs['error_msg'] = dfs['error_msg'].apply(t.tokenize)

In [ ]:
dfs.to_hdf('tokenized.h5', 'frame')

In [12]:
def get_frequencies(df, col_name):
    # Count the word occurences
    str_frequencies = pd.DataFrame(list(Counter(filter(None,list(itertools.chain(*df[col_name]
                                                                            )))).items()),columns=['word','count'])
    return str_frequencies.sort_values(by ='count',ascending=False  )

In [13]:
def get_counts(df, col_name):
    
    # Count the word occurences
    str_frequencies = Counter(filter(None,list(itertools.chain(*df[col_name]))))
    return str_frequencies  

In [14]:
str_frequencies = get_frequencies(dfs, 'error_msg')

In [15]:
str_frequencies.head()

,word,count
66007,:,7087727
739312,",",3846930
298060,--,3359043
253406,>,3305914
159612,<,3128277


In [15]:
minimum_count = 10
low_frequency_words = set(str_frequencies[str_frequencies['count'] < minimum_count]['word'])

In [16]:
import string 
# Remove punctuation and kick out words that occur less than a certain threshold
def check_first_and_last_char(word):
    if word[0] in "':/.-": 
        #print word
        word = word.replace(word[0], '')
        #print word
    if len(word) > 1:
        if word[-1] in "':/.-": 
            word = word.replace(word[-1], '')
    return word

# Remove punctuation and kick out words that occur less than a certain threshold
def clean_data(tokens):
    words = filter(lambda word: word not in '``#\'"\'\'|==,|--$;:=+><[!@]|&?{}...%(.)""()==========' and word not in low_frequency_words, tokens)
    words_cleaned = [check_first_and_last_char(word) for word in words]
    #words_cleaned = filter(lambda word: word.isalpha() == False, words_cleaned)
    words_cleaned = filter(lambda word: word not in 'abcdefghijklmnopqrstuvwxyzn=', words_cleaned)
    return words_cleaned

In [17]:
dfs['error_msg'] = dfs['error_msg'].apply(clean_data)
#res = test['tokenized_text_filtered'].agg('sum')

In [14]:
tokens = df_msg.iloc[207]
print 'Original message'
print
print tokens["error_msg"]
print
"""
print 'Tokenized'
print
print tokens["tokenized_text"]
print
"""
print 'Filtered'
print
print tokens["tokenized_text_filtered"]

Original message

Adding last 25 lines of CMSSW stdout:   734 PFEGammaAlgo::mergeROsByAnyLink 1/115407         1/115529         1/118126   735 PFTrackTransformer   1/115277         1/115277         1/118020   736 PFTrackTransformer   1/115277         1/115277         1/118020   737 RecoMuon             1/115516         1/117754         1/118143   738 XrdAdaptor           pre-events                           739 XrdAdaptorInternal   pre-events       pre-events       pre-events   740 XrdFileWarning       pre-events                           741 FailedPropagation    1/115534                             742 TimeReport           PostGlobalEndRun                     743 TrajectoryNotPosDef  1/115282                             744 Fatal Exception      AfterSource                          745 Fatal Exception      1/118162         1/118154         1/118166   746 MemoryReport         PostGlobalEndRun                     747 fileAction           PostGlobalEndRun                     748 fileActio

In [18]:
str_frequencies_filtered = get_frequencies(dfs, 'error_msg')

In [19]:
str_frequencies_filtered.head()

,word,count
10294,edm,637270
12853,in,604719
21897,file,501612
23557,1,417321
10507,is,402361


In [ ]:
dfs.to_hdf('dfs_tokenized.h5', 'frame')

In [23]:
print len(str_frequencies_filtered)

33369


In [ ]:
MAX_NB_WORDS = len(str_frequencies_filtered)
MAX_SEQUENCE_LENGTH = 5000

In [ ]:
word_index = {t: i+1 for i,t in enumerate(str_frequencies_filtered['word'])}

In [ ]:
def encode(tokens):
    return [word_index[w] for w in tokens]

In [ ]:
df_msg['encoded_tokens'] = df_msg['tokenized_text_filtered'].apply(encode)

In [ ]:
tokens = df_msg.iloc[207]
print
print tokens["encoded_tokens"]
print

In [ ]:
pd.set_option('display.max_rows', 100)
str_frequencies_filtered.head(100)

In [ ]:
most_freq = str_frequencies_filtered['word'][0:100]

In [ ]:
out = df_msg.drop([ 'error_msg', 'side_state','action', 'memory', 'tokenized_text', 'tokenized_text_filtered' ], axis = 1)

In [ ]:
out.head()

In [ ]:
! pwd

In [ ]:
import pickle
with open('filtered_messages.pkl', 'wb') as f:
    pickle.dump(out, f)

## 2. Analyze the filtered words

In [ ]:
counts = get_counts(df_msg, 'tokenized_text_filtered')

In [ ]:
type(counts)

In [ ]:
import wordcloud
import matplotlib.pyplot as plt

In [ ]:
wc = wordcloud.WordCloud(max_words=100, background_color="white", width=1200, height=800).generate_from_frequencies(counts)

In [ ]:
def random_color_func(word=None, font_size=None, position=None,  orientation=None, font_path=None, random_state=None):
    h = int(360.0 * random_state.randint(0, 1000) / 255.0)
    s = int(100.0 * 255.0 / 255.0)
    l = int(100.0 * float(random_state.randint(60, 120)) / 255.0)

    return "hsl({}, {}%, {}%)".format(h, s, l)

In [ ]:
wordcloud = wordcloud.WordCloud(width=1600, height=800, color_func=random_color_func, background_color="white", max_words=100).generate_from_frequencies(counts)

In [ ]:
# Save the image in the img folder:
wordcloud.to_file("img/first_review.png")

In [ ]:
#plt.figure( figsize=(20,10), facecolor='k')
plt.figure( figsize=(24,14), dpi=1200)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.tight_layout(pad=0)
plt.savefig('wordcloud.png', facecolor='k', bbox_inches='tight')

In [ ]:
import matplotlib.pyplot as plt
ax = df_msg['tokenized_text_filtered'].apply(lambda x: len(x)).hist(bins = 1000)
plt.xlim([0,10000])
ax.set_yscale('log')
ax.set_xlabel('Number of words')
ax.set_ylabel('Number of workflows')

## 3.0. Load the google model and inspect

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
model = KeyedVectors.load_word2vec_format('..//..//GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
words_in = 0
words_out = 0
for w in list(str_frequencies_filtered['word']):
    if w not in model:
        words_out += 1
        #print w
    else:
        words_in += 1
print words_in, words_out

In [ ]:
# Number of tokens:
print 'Number of tokens', len(list(str_frequencies['word']))
print 'Number of tokens filtered', len(list(str_frequencies_filtered['word']))
print 'Fraction in Google News Model', "%.2f" % (float(words_in) / float(words_in + words_out))

In [ ]:
words_in = 0
words_out = 0
words = [word.lower() for word in list(str_frequencies_filtered['word'])]
print words[0:100]

In [ ]:
import pickle
with open('words.pkl', 'wb') as f:
    pickle.dump(words, f)

In [ ]:
for w in set(list(words)):
    if w not in model:
        words_out += 1
        #print w
    else:
        words_in += 1
print words_in, words_out

## 3. Train the word2vec model and visualize with tsne

In [21]:
texts_stemmed = list(dfs['error_msg'])
w2v_sg = gensim.models.Word2Vec(texts_stemmed, size=50, window=5, min_count=5, workers=4, sg=1)
#w2v_cbow = gensim.models.Word2Vec(texts_stemmed, size=50, window=5, min_count=5, workers=4, sg=0)

In [22]:
w2v_sg.save("word2vec_extended.model")

In [21]:
! pwd

/eos/home-l/llayer/AIErrorLogAnalysis/spark


In [23]:
word2index = {token: token_index for token_index, token in enumerate(w2v_sg.wv.index2word)}

In [24]:
print len(word2index)

33369


In [25]:
def encode(tokens):
    return [word2index[w] for w in tokens]

In [28]:
dfs['error_msg'] = dfs['error_msg'].apply(encode)

In [30]:
dfs.to_hdf('messages_tokenized_new.h5', 'frame')

/cvmfs/sft-nightlies.cern.ch/lcg/views/dev3/Mon/x86_64-centos7-gcc8-opt/lib/python2.7/site-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['task_name', 'site', 'error_msg', 'error_type', 'names']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [21]:
out = df_msg.drop([ 'error_msg', 'side_state','action', 'memory', 'tokenized_text', 'tokenized_text_filtered' ], axis = 1)

In [22]:
list(out)

['task_name', 'site', 'error', 'encoded_tokens']

In [25]:
out.site = out.site.str.encode('utf-8')

In [26]:
list(out)

['task_name', 'site', 'error', 'encoded_tokens']

In [27]:
import pickle
with open('w2v_messages.pkl', 'wb') as f:
    pickle.dump(out, f)

In [30]:
out.to_hdf('w2v_messages.h5', 'frame')

/cvmfs/sft.cern.ch/lcg/views/LCG_95a/x86_64-centos7-gcc7-opt/lib/python2.7/site-packages/pandas/core/generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['task_name', 'site', 'encoded_tokens']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [29]:
! pip install tables --user

    100% |████████████████████████████████| 4.3MB 5.1MB/s eta 0:00:01
tensorflow 1.12.0 requires backports.weakref>=1.0rc1, which is not installed.
wordcloud 1.5.0 requires pillow, which is not installed.
py2neo 4.0.0 requires neo4j-driver>=1.6.0, which is not installed.
blis 0.2.4 has requirement numpy>=1.15.0, but you'll have numpy 1.14.2 which is incompatible.
keras 2.2.0 has requirement keras_applications==1.0.2, but you'll have keras-applications 1.0.6 which is incompatible.
keras 2.2.0 has requirement keras_preprocessing==1.0.1, but you'll have keras-preprocessing 1.0.5 which is incompatible.
  The scripts pt2to3, ptdump, ptrepack and pttree are installed in '/eos/user/l/llayer/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You are using pip version 10.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [31]:
embedding_matrix = np.zeros((len(word2index)+1, 50))
for word, i in word2index.items():
    embedding_matrix[i] = w2v_sg.wv[word]

In [32]:
print embedding_matrix.shape

(33370, 50)


In [34]:
import pickle
with open('embedding_matrix_new.pkl', 'wb') as f:
    pickle.dump(embedding_matrix, f)

In [ ]:
embeddi

In [30]:
! ls

create_actionshistory_keys.ipynb  number_errors.png	w2v_messages.pkl
create_embeddings.ipynb		  sites.png		word2vec.model
errors.png			  train_word2vec.ipynb	wordcloud.png
filtered_messages.pkl		  Untitled.ipynb	words.pkl
filtered_msg.csv		  utils.py
filter_wmarchive.ipynb		  w2v.csv


In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [ ]:
def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in most_freq:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(10, 10)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()
    

### Analyze skip-gram

In [ ]:
tsne_plot(w2v_sg)

In [ ]:
df_msg.head()

In [ ]:
# Get TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
def dummy(doc):
    return doc
tfidf = TfidfVectorizer(
    tokenizer=dummy,
    preprocessor=dummy,)
x = tfidf.fit_transform(df_msg['tokenized_text_filtered'])
word_weights = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

In [ ]:
def get_average_vector(text):
    return np.mean(np.array([w2vmodel_stemmed.wv[w] for w in text if w in w2vmodel_stemmed]), axis=0)

In [ ]:
def get_average_vector_tfidf(text): 
    return np.mean(np.array([w2vmodel_stemmed.wv[w] * word_weights[w] for w in text if w in w2vmodel_stemmed]), axis=0)

In [ ]:
df_msg['w2v_tfidf'] = df_msg['tokenized_text_filtered'].apply(get_average_vector_tfidf)

In [ ]:
df_msg['w2v'] = df_msg['tokenized_text_filtered'].apply(get_average_vector)

In [ ]:
type(text_vecs)

In [ ]:
clone_docs = df_msg[(df_msg['action'] == 'clone')].sample(3000)
acdc_docs = df_msg[(df_msg['action'] == 'acdc')].sample(3000)

In [ ]:
test_docs = pd.concat([clone_docs, acdc_docs])

In [ ]:
print len(test_docs)

In [ ]:
import matplotlib.cm as cm

In [ ]:
def tsne_plot_classes(model, df, col):
    "Creates and TSNE model and plots it"

    avg_vec = list(df[col])
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(list(avg_vec))

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
           
    return x,y

In [ ]:
x_w2v_sg, y_w2v_sg = tsne_plot_classes(w2v_sg, test_docs, 'w2v')

In [ ]:
x_w2v_tfidf_sg, y_w2v_tfidf_sg = tsne_plot_classes(w2v_sg, test_docs, 'w2v_tfidf')

In [ ]:
def plot_tsne(df, x, y, title):
    
    labels = df['action']
    error_codes = df['error']
    error_codes_unique = list(df.groupby('error').size().reset_index(name='size').sort_values(by ='size',ascending=False  )['error'][0:10])
    error_colors = {}
    for counter, code in enumerate(error_codes_unique):
        error_colors[code] = counter
    print error_colors
    
    colors = cm.rainbow(np.linspace(0, 1, len(error_codes_unique)))    
    
    print len(colors), len(error_codes_unique)
    plt.figure(figsize=(10, 10)) 
    
    legend = []
    for i in range(len(x)):

        error_code = error_codes.iloc[i]
        if error_code not in error_codes_unique:
            continue
        color = error_colors[error_code]
        if error_code not in legend:
            #print error_code, color
            plt.scatter(x[i],y[i], c=colors[color], label = str(error_code))
            legend.append(error_code)
        else:
            plt.scatter(x[i],y[i], c=colors[color])

        """
        if labels.iloc[i] == 'acdc':
            plt.scatter(x[i],y[i], c='red')
        else:
            plt.scatter(x[i],y[i], c='blue')

        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
        """
        
    
    plt.legend(loc='upper right')   
    plt.title(title)
        
    
    plt.show()

In [ ]:
plot_tsne(test_docs, x_w2v_sg, y_w2v_sg, 'Word2Vec Skip-Gram 100d averaged')

In [ ]:
plot_tsne(test_docs, x_w2v_tfidf_sg, y_w2v_tfidf_sg, 'Word2Vec Skip-Gram 100d tfidf averaged')

In [ ]:
tsne_plot_classes(w2vmodel_stemmed, test_docs, 10, )

In [ ]:
tsne_plot_classes(w2vmodel_stemmed, test_docs, 10)

## 4. Save everything

In [ ]:
list(df_msg)

In [ ]:
out = df_msg.drop([ 'error_msg', 'side_state','action', 'memory', 'tokenized_text', 'tokenized_text_filtered' ], axis = 1)

In [ ]:
def array_to_string(my_list):
    return '[' + ','.join([str(elem) for elem in my_list]) + ']'

In [ ]:
out["w2v"]= out["w2v"].apply(array_to_string)

In [ ]:
out.to_csv('w2v.csv')

In [ ]:
out.head()